# Initializing the EMMAA Neurofibromatosis Model

In this notebook we illustrate how we put together an initial neurofibromatosis-specific knowledge network based on molecular interactions we have compiled from reading all available abstracts and full text articles from PubMed.

As background, we use multiple text mining systems from our collaborators and from the research community to extract raw causal relations and then assemble them using our knowledge assembly system, INDRA (the Integrated Network and Dynamical Reasoning Assembler). We do this systematically across all available literature and store the results in the INDRA Database for use in downstream projects like this one. 

The first step in creating the EMMAA model for Neurofibromatosis is to find publications broadly relevant for neurofibromatosis. We do this by querying based on the both the term "neurofibromatosis" as well as the top-level Medical Subject Heading (MESH) term for Neurofibromatoses, D017253 (see https://www.ncbi.nlm.nih.gov/mesh/68017253). We use the `indra.literature` module to search PubMed using the PubMed REST API.

In [21]:
# Preliminaries
import time
import pickle
from collections import Counter
from indra_db import get_primary_db
from indra.util import batch_iter
from indra_db.util import distill_stmts
from indra.tools import assemble_corpus as ac
from indra.sources import indra_db_rest as idr


In [22]:
# Query PubMed for articles related to NF
def get_nf_pmids():
    # PMIDs from text search for "neurofibromatosis"
    nf_text_pmids = pubmed_client.get_ids('neurofibromatosis')
    # Mesh term for neurofibromatoses
    nf_mesh_ids = pubmed_client.get_ids_for_mesh('D017253')
    return list(set(nf_text_pmids + nf_mesh_ids))

# Get the PMIDs
nf_pmids = get_nf_pmids()

Here's how many unique PMIDs we found:

In [24]:
len(nf_pmids)

17879

The next step is to get the INDRA Statements that have been extracted from these and stored in the INDRA Database, which will give us a sense of which molecular entities (proteins, drugs, etc.) are relevant to NF and should be included in the model.

In [39]:
stmts = []
batch_size = 100
total_batches = int(len(nf_pmids) / batch_size) + 1
for batch_ix, pmid_batch in enumerate(batch_iter(nf_pmids, 100)):
    print(batch_ix + 1, "of", total_batches)
    idrp = idr.get_statements_for_paper([('pmid', pmid) for pmid in pmid_batch])
    stmts.extend(idrp.statements)
#for idrp = idr.get_statements_for_paper([('pmid', pmid) for pmid in nf_pmids][0:100])

INFO: [2020-11-11 08:49:29] indra.sources.indra_db_rest.util - query: https://db.indra.bio/statements/from_papers
INFO: [2020-11-11 08:49:29] indra.sources.indra_db_rest.util - params: {'ev_limit': 10, 'best_first': True, 'max_stmts': None, 'api_key': '[api-key]'}


1 of 179


INFO: [2020-11-11 08:49:31] indra.sources.indra_db_rest.util - query: https://db.indra.bio/statements/from_papers
INFO: [2020-11-11 08:49:31] indra.sources.indra_db_rest.util - params: {'ev_limit': 10, 'best_first': True, 'max_stmts': None, 'api_key': '[api-key]'}


2 of 179


INFO: [2020-11-11 08:49:32] indra.sources.indra_db_rest.util - query: https://db.indra.bio/statements/from_papers
INFO: [2020-11-11 08:49:32] indra.sources.indra_db_rest.util - params: {'ev_limit': 10, 'best_first': True, 'max_stmts': None, 'api_key': '[api-key]'}


...

177 of 179


INFO: [2020-11-11 09:06:09] indra.sources.indra_db_rest.util - query: https://db.indra.bio/statements/from_papers
INFO: [2020-11-11 09:06:09] indra.sources.indra_db_rest.util - params: {'ev_limit': 10, 'best_first': True, 'max_stmts': None, 'api_key': '[api-key]'}


178 of 179


INFO: [2020-11-11 09:06:18] indra.sources.indra_db_rest.util - query: https://db.indra.bio/statements/from_papers
INFO: [2020-11-11 09:06:18] indra.sources.indra_db_rest.util - params: {'ev_limit': 10, 'best_first': True, 'max_stmts': None, 'api_key': '[api-key]'}


179 of 179


In [40]:
len(stmts)

14334

In [4]:
# Get raw statement IDs from the DB for the given TextRefs
reload = False
pmid_stmts_file = 'nf_raw_stmts.pkl'
if reload: 
    db = get_primary_db()
    # Get statements for the given text refs
    print(f"Distilling statements for {len(nf_pmids)} TextRefs")
    start = time.time()
    clauses = [
        db.TextRef.pmid.in_(nf_pmids),
        db.TextContent.text_ref_id == db.TextRef.id,
        db.Reading.text_content_id == db.TextContent.id,
        db.RawStatements.reading_id == db.Reading.id]
    pmid_stmts = distill_stmts(db, get_full_stmts=True, clauses=clauses)
    end = time.time()
    elapsed = end - start
    print(elapsed)
    list(ac.dump_statements(pmid_stmts, pmid_stmts_file))
else:
    pmid_stmts = list(ac.load_statements(pmid_stmts_file))


INFO: [2020-11-10 15:46:37] indra.tools.assemble_corpus - Loading nf_raw_stmts.pkl...
INFO: [2020-11-10 15:46:38] indra.tools.assemble_corpus - Loaded 20209 statements


In [20]:
[s for s in pmid_stmts if len(s.agent_list()) == 2 and None not in s.agent_list()
                                             and s.agent_list()[0].name.startswith('ITG')
                                             and s.agent_list()[1].name == 'PAK']

[Activation(ITGA5(), PAK()),
 Activation(ITGA5(), PAK()),
 Activation(ITGA5(), PAK()),
 Activation(ITGA5(), PAK())]

In [5]:
proteins = [ag.name for stmt in pmid_stmts for ag in stmt.agent_list()
            if ag is not None and ('HGNC' in ag.db_refs or 'FPLX' in ag.db_refs)]
protein_ctr = Counter(proteins)
protein_ctr = sorted([(k, ct) for k, ct in protein_ctr.items()], key=lambda x: x[1], reverse=True)

In [6]:
protein_ctr

[('NF2', 2843),
 ('NF1', 2665),
 ('RAS', 1147),
 ('ERK', 719),
 ('AKT', 309),
 ('EGFR', 292),
 ('MTOR', 276),
 ('DPYSL2', 252),
 ('RAC', 233),
 ('TP53', 209),
 ('YAP1', 206),
 ('PI3K', 198),
 ('PAK', 179),
 ('mTORC1', 178),
 ('RASA1', 176),
 ('STAT3', 176),
 ('MEK', 165),
 ('SPRED1', 142),
 ('RAF', 140),
 ('AKT1', 138),
 ('CAMP', 133),
 ('CTNNB1', 130),
 ('PCNA', 129),
 ('BRAF', 127),
 ('RAC1', 126),
 ('PAK1', 124),
 ('DCAF1', 118),
 ('CDKN1A', 116),
 ('CD44', 116),
 ('LIMK2', 116),
 ('PTK2', 109),
 ('CDC42', 108),
 ('PXN', 106),
 ('Cofilin', 106),
 ('LATS1', 102),
 ('TSC2', 98),
 ('ERBB2', 97),
 ('HRAS', 97),
 ('MAP3K11', 95),
 ('RAF1', 90),
 ('Actin', 86),
 ('PKA', 83),
 ('KRAS', 80),
 ('SRC', 78),
 ('ETV5', 76),
 ('RasGAP', 76),
 ('EGF', 74),
 ('JNK', 74),
 ('AMOT', 70),
 ('TGFB', 69),
 ('ALK', 66),
 ('SLC9A3R1', 64),
 ('PTEN', 62),
 ('EZR', 61),
 ('MAPK', 56),
 ('Wnt', 55),
 ('GRB10', 54),
 ('KIT', 54),
 ('TSC1', 54),
 ('VEGFA', 52),
 ('TNFRSF11B', 50),
 ('GFER', 49),
 ('MLN', 49),

In [14]:
len(protein_ctr)

1538

In [7]:
ag_stmts = [s for s in pmid_stmts if 'Integrins' in [ag.name for ag in s.agent_list() if ag is not None]]

In [8]:
ag_stmts.sort(key=lambda x: len(x.evidence), reverse=True)

In [9]:
ag_stmts

[Activation(Integrins(), mitogen activated protein kinase()),
 Activation(Integrins(), FAK()),
 Complex(NF2(), Integrins()),
 Complex(Integrins(), ERBB2(), ERBB3()),
 Complex(NF2(), Integrins()),
 Activation(Integrins(), ERBB2()),
 Complex(FN1(), Integrins()),
 Activation(Integrins(), NF2()),
 Complex(NF2(), Integrins()),
 Complex(NF2(), RTK(), Integrins()),
 Activation(Integrins(), mitogen activated protein kinase()),
 Complex(growth factor receptor(), Integrins()),
 Complex(Integrins(), Actin()),
 Complex(PXN(), Integrins()),
 Complex(Integrins(), RTK()),
 Complex(Integrins(), LAYN()),
 Complex(Integrins(), LAMB1()),
 Complex(NF1(), Integrins(), intermediate filament()),
 Complex(Integrins(), PXN()),
 Activation(Integrins(), PAK()),
 Complex(PXN(), Integrins()),
 Activation(Integrins(), mTORC1()),
 Complex(NF2(), Integrins()),
 Complex(Integrins(), ADAM15()),
 Complex(EGFR(), ERBB3(), ERBB4(), Integrins()),
 Complex(NF2(), Integrins()),
 Complex(DEFB1(), Integrins(), ERBB2()),
 Activ

Get drugs that:
1. Have already been tested against NF (are in the screening datasets)
2. Are known to target one or more genes relevant to NF.

# Drug screening dataset

In [16]:
import synapseclient
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns

In [9]:
syn = synapseclient.Synapse()
syn.login()

Welcome, johnbachman!



In [20]:
#from Synapse-stored csv
data_path = syn.get("syn20682897").path
data = pd.read_csv(data_path, low_memory=False)   
#from Synapse table
#results = syn.tableQuery("SELECT * FROM syn20556247")
#data = results.asDataFrame()


In [28]:
data.head()

,drug_screen_id,drug_assay_id,experiment_synapse_id,study_synapse_id,funder,model_name,cellosaurus_id,organism_name,drug_name,DT_explorer_internal_id,dosage,dosage_unit,response,response_type,response_unit,model_type,disease_name,disease_efo_id,symptom_name,symptom_efo_id
0,1,syn11373153.17,syn11373153.1,syn5610425,CTF,N10,NaN,human,GPHR-00000018,159129,100.000000,uM,99.979123,percent viability,%,cell line,no disease,NaN,no symptom,NaN
1,1,syn11373153.18,syn11373153.1,syn5610425,CTF,N10,NaN,human,GPHR-00000018,159129,33.333430,uM,103.789330,percent viability,%,cell line,no disease,NaN,no symptom,NaN
2,1,syn11373153.19,syn11373153.1,syn5610425,CTF,N10,NaN,human,GPHR-00000018,159129,11.111175,uM,102.178887,percent viability,%,cell line,no disease,NaN,no symptom,NaN
3,1,syn11373153.20,syn11373153.1,syn5610425,CTF,N10,NaN,human,GPHR-00000018,159129,3.703736,uM,104.653527,percent viability,%,cell line,no disease,NaN,no symptom,NaN
4,1,syn11373153.21,syn11373153.1,syn5610425,CTF,N10,NaN,human,GPHR-00000018,159129,1.234554,uM,105.312040,percent viability,%,cell line,no disease,NaN,no symptom,NaN


## Summary data

In [97]:
data_path = syn.get("syn20684161").path
data = pd.read_csv(data_path, low_memory=False)   
data.response_type.unique()

array(['AUC_Simpson', 'AUC_Trapezoid', 'IC50_abs', 'Min_viability'],
      dtype=object)

In [98]:
ic50 = data[data.response_type == 'Min_viability']

In [100]:
data.disease_name.unique()

array(['no disease', 'NF1', 'NF2'], dtype=object)

In [101]:
targetspath = syn.get('syn17091507')
readRDS = robjects.r['readRDS']
targets = readRDS(targetspath.path)

In [81]:
targets_filt = (targets
                .query('mean_pchembl > 6')
                .filter(["internal_id", "hugo_gene", "std_name"])
                .drop_duplicates())

In [88]:
targets.head()

,internal_id,hugo_gene,n_quantitative,mean_pchembl,cv,sd,IC50_nM,AC50_nM,EC50_nM,Potency_nM,Ki_nM,Kd_nM,n_qualitative,std_name,total_n,confidence,pchembl_d,pchembl_t,known_selectivity_index
1,3,HTR7,1,8.01,NaN,NaN,NaN,NaN,NaN,NaN,9.7,NaN,-2147483648,CHEMBL2413451,1,-0.229,8.01,10322.90,1.0
2,4,CHRNA4,1,7.54,NaN,NaN,NaN,NaN,NaN,NaN,29.0,NaN,-2147483648,CHEMBL204871,1,-0.229,15.08,4501.97,0.5
3,4,CHRNB2,1,7.54,NaN,NaN,NaN,NaN,NaN,NaN,29.0,NaN,-2147483648,CHEMBL204871,1,-0.229,15.08,4194.75,0.5
4,5,GSK3A,2,7.76,5.285076,0.410122,34.0,NaN,NaN,NaN,9.0,NaN,-2147483648,CHEMBL3582401,2,0.393,7.76,2034.63,1.0
5,6,FAAH,1,9.30,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,-2147483648,CHEMBL2386554,1,-0.229,9.30,6375.76,1.0


In [83]:
data_tgts = ic50.merge(targets, left_on="DT_explorer_internal_id",
                    right_on="internal_id",
                    how="inner")
#.filter(["DT_explorer_internal_id", "hugo_gene"]
#                                       ).assign(foo = 1
#                        ).pivot_table(values = 'foo',
#                                      index = 'DT_explorer_internal_id',
#                                      columns = "hugo_gene", fill_value = 0)"""

In [90]:
data_tgts = ic50.merge(targets, left_on="DT_explorer_internal_id",
                    right_on="internal_id",
                    how="inner")
data_tgts = data_tgts[['disease_name', 'DT_explorer_internal_id',
           'drug_name', 'std_name', 'dosage', 'response_type',
           'response', 'response_unit',
           'hugo_gene', 'mean_pchembl', 'cv', 'sd', 'IC50_nM', 
           'AC50_nM', 'EC50_nM', 'known_selectivity_index']]
data_tgts = data_tgts[(data_tgts.response != 0) & (data_tgts.response != np.inf)]

In [96]:
data_tgts[data_tgts.std_name == 'TEMSIROLIMUS'].iloc[0:50]

,disease_name,DT_explorer_internal_id,drug_name,std_name,dosage,response_type,response,response_unit,hugo_gene,mean_pchembl,cv,sd,IC50_nM,AC50_nM,EC50_nM,known_selectivity_index
75204,NF1,179480,NCGC00167518-03,TEMSIROLIMUS,"[0.000780415,46.08294931]",IC50_abs,1.392725e+03,uM,MTOR,5.75,NaN,NaN,1760.0,NaN,NaN,1.0
75205,NF1,179480,NCGC00167518-03,TEMSIROLIMUS,"[0.000780415,46.08294931]",IC50_abs,1.392725e+03,uM,BRAF,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75206,NF1,179480,NCGC00167518-03,TEMSIROLIMUS,"[0.000780415,46.08294931]",IC50_abs,6.024084e+01,uM,MTOR,5.75,NaN,NaN,1760.0,NaN,NaN,1.0
75207,NF1,179480,NCGC00167518-03,TEMSIROLIMUS,"[0.000780415,46.08294931]",IC50_abs,6.024084e+01,uM,BRAF,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75208,no disease,179480,NCGC00167518-03,TEMSIROLIMUS,"[0.00078041474654,46.082949308]",IC50_abs,3.092603e+09,uM,MTOR,5.75,NaN,NaN,1760.0,NaN,NaN,1.0
75209,no disease,179480,NCGC00167518-03,TEMSIROLIMUS,"[0.00078041474654,46.082949308]",IC50_abs,3.092603e+09,uM,BRAF,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75210,NF1,179480,NCGC00167518-03,TEMSIROLIMUS,"[7.8e-10,4.61e-05]",IC50_abs,3.287926e-05,uM,MTOR,5.75,NaN,NaN,1760.0,NaN,NaN,1.0
75211,NF1,179480,NCGC00167518-03,TEMSIROLIMUS,"[7.8e-10,4.61e-05]",IC50_abs,3.287926e-05,uM,BRAF,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Notebook for old dataset

In [11]:
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()


In [12]:
targetspath = syn.get('syn17091507')
readRDS = robjects.r['readRDS']

targets = readRDS(targetspath.path)

In [15]:

targets_filt = (targets
                .query('mean_pchembl > 6')
                .filter(["internal_id", "hugo_gene", "std_name"])
                .drop_duplicates())
    
pd.DataFrame.head(targets_filt)

,internal_id,hugo_gene,std_name
1,3,HTR7,CHEMBL2413451
2,4,CHRNA4,CHEMBL204871
3,4,CHRNB2,CHEMBL204871
4,5,GSK3A,CHEMBL3582401
5,6,FAAH,CHEMBL2386554


In [17]:
drug_data_path = syn.get("syn17462699").path
drug_data = pd.read_csv(drug_data_path, low_memory=False)
                       
pd.DataFrame.head(drug_data)

SynapseHTTPError: 403 Client Error: 
You do not have READ permission for the requested entity, syn17462699.